<a href="https://colab.research.google.com/github/maskot1977/tmd2022/blob/jAYCHc8S/tmd2022_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

「AI創薬・ケモインフォマティクス入門」講義資料　（講師：小寺正明）

3月11日（土）19:40～21:10　第5回  「計算機実験1」

# RDKit インストール

In [ ]:
!pip install rdkit-pypi

# 化合物データ取得

In [ ]:
import pandas as pd

# csvからのデータ読み込み_
url = "https://raw.githubusercontent.com/maskot1977/toydata/main/data/data_18.csv"
database1 = pd.read_csv(url)
database1

# 回帰問題用目的変数とその分布

In [ ]:
import matplotlib.pyplot as plt

plt.title("Melting point as continus value Y1")
Y1 = database1["Melting point"]
Y1.hist(bins=20)
plt.show()

# 分類用目的変数とその分布（練習のため）

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Y2 = pd.DataFrame(
    np.where(
        database1["Melting point"]
        > database1["Melting point"].describe().median() * 0.9,
        1,
        0,
    ),
    columns=["Melting point as discrete value Y2"],
)
Y2.hist(bins=20)
plt.show()

# バイアスのある分類用目的変数とその分布（練習のため）

In [ ]:
import numpy as np

Y3 = pd.DataFrame(
    np.where(
        database1["Melting point"]
        > database1["Melting point"].describe().median() * 2.2,
        1,
        0,
    ),
    columns=["Melting point as biased discrete value Y3"],
)
Y3.hist(bins=20)
plt.show()

# RDKit supporter

RDKit supporter は、RDKit や ML 周りで便利な関数やクラスを私が書き溜めたものです。インストールしてみましょう。

In [ ]:
!pip install git+https://github.com/maskot1977/rdkit_supporter.git

# RDKit 記述子

In [ ]:
%%time
import rdkit
from rdkit_supporter.descriptors import calc_descriptors

rdkit_df = calc_descriptors(database1["Open Babel SMILES"])
display(rdkit_df)

# 説明変数

今回は、説明変数 X として RDKit descriptors を用います。

In [ ]:
X = rdkit_df

# 欠損値の補間の例

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer()
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# 多様体学習 (Manifold Learning)

## PCA

In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

manifold = PCA()
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet")
plt.xlabel("PC1({:.2f}%)".format(manifold.explained_variance_ratio_[0] * 100))
plt.ylabel("PC2({:.2f}%)".format(manifold.explained_variance_ratio_[1] * 100))
plt.colorbar()
plt.show()

## 前処理

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

manifold = Pipeline([("scaler", StandardScaler()), ("pca", PCA())])
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet")
plt.xlabel("PC1({:.2f}%)".format(manifold["pca"].explained_variance_ratio_[0] * 100))
plt.ylabel("PC2({:.2f}%)".format(manifold["pca"].explained_variance_ratio_[1] * 100))
plt.colorbar()
plt.show()

## 目的変数で色付け

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

manifold = Pipeline([("scaler", StandardScaler()), ("pca", PCA())])
embedding = manifold.fit_transform(X)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axes[0].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet")
axes[1].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y2.values.flatten())
axes[2].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y3.values.flatten())
axes[0].set_xlabel(
    "PC1({:.2f}%)".format(manifold["pca"].explained_variance_ratio_[0] * 100)
)
axes[1].set_xlabel(
    "PC1({:.2f}%)".format(manifold["pca"].explained_variance_ratio_[0] * 100)
)
axes[2].set_xlabel(
    "PC1({:.2f}%)".format(manifold["pca"].explained_variance_ratio_[0] * 100)
)
axes[0].set_ylabel(
    "PC2({:.2f}%)".format(manifold["pca"].explained_variance_ratio_[1] * 100)
)
plt.show()

# t-SNE

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

manifold = Pipeline([("scaler", StandardScaler()), ("tsne", TSNE())])
embedding = manifold.fit_transform(X)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axes[0].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet")
axes[1].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y2.values.flatten())
axes[2].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y3.values.flatten())
plt.show()

## t-SNE はパラメータによって形が大きく変わってしまう

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))
for i, early_exaggeration in enumerate((5, 30, 50)):
    for j, perplexity in enumerate((5, 30, 50)):
        manifold = Pipeline(
            [
                ("scaler", StandardScaler()),
                (
                    "tsne",
                    TSNE(early_exaggeration=early_exaggeration, perplexity=perplexity),
                ),
            ]
        )
        embedding = manifold.fit_transform(X)
        axes[i][j].set_title(
            "early_exaggeration={} perplexity={}".format(early_exaggeration, perplexity)
        )
        axes[i][j].scatter(
            embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet"
        )
plt.show()

## Isomap

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

manifold = Pipeline([("scaler", StandardScaler()), ("isomap", Isomap())])
embedding = manifold.fit_transform(X)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axes[0].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet")
axes[1].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y2.values.flatten())
axes[2].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y3.values.flatten())
plt.show()

## Isomap もまたパラメータによって形が大きく変わってしまう

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))
for i, algorithm in enumerate(["brute", "kd_tree", "ball_tree"]):
    for j, n_neighbors in enumerate((5, 30, 50)):
        manifold = Pipeline(
            [
                ("scaler", StandardScaler()),
                (
                    "isomap",
                    Isomap(neighbors_algorithm=algorithm, n_neighbors=n_neighbors),
                ),
            ]
        )
        embedding = manifold.fit_transform(X)
        axes[i][j].set_title(
            "algorithm={} n_neighbors={}".format(
                algorithm,
                n_neighbors,
            )
        )
        axes[i][j].scatter(
            embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet"
        )
plt.show()

## UMAP

In [ ]:
!pip install umap-learn

In [ ]:
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from umap import UMAP

manifold = Pipeline([("scaler", StandardScaler()), ("umap", UMAP())])
embedding = manifold.fit_transform(X)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axes[0].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet")
axes[1].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y2.values.flatten())
axes[2].scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y3.values.flatten())
plt.show()

# UMAP もまたパラメータによって形が大きく変わってしまう

In [ ]:
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from umap import UMAP

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))
for i, min_dist in enumerate((0.1, 0.5, 0.9)):
    for j, n_neighbors in enumerate((5, 30, 50)):
        manifold = Pipeline(
            [
                ("scaler", StandardScaler()),
                ("umap", UMAP(min_dist=min_dist, n_neighbors=n_neighbors)),
            ]
        )
        embedding = manifold.fit_transform(X)
        axes[i][j].set_title("min_dist={} n_neighbors={}".format(min_dist, n_neighbors))
        axes[i][j].scatter(
            embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y1, cmap="jet"
        )
plt.show()

# データ分割

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y1_train, Y1_test = train_test_split(
    X, Y1, test_size=0.5, random_state=53
)

In [ ]:
plt.hist(Y1_train, alpha=0.5, label="training data")
plt.hist(Y1_test, alpha=0.5, label="test data")
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y2_train, Y2_test = train_test_split(
    X, Y2, test_size=0.5, random_state=53
)

In [ ]:
plt.hist(Y2_train, alpha=0.5, label="training data")
plt.hist(Y2_test, alpha=0.5, label="test data")
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y3_train, Y3_test = train_test_split(
    X, Y3, test_size=0.5, random_state=53, stratify=Y3
)

In [ ]:
plt.hist(Y3_train, alpha=0.5, label="training data")
plt.hist(Y3_test, alpha=0.5, label="test data")
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

(
    X_train,
    X_test,
    Y1_train,
    Y1_test,
    Y2_train,
    Y2_test,
    Y3_train,
    Y3_test,
) = train_test_split(X, Y1, Y2, Y3, test_size=0.5, random_state=53, stratify=Y3)

# 様々な回帰モデルで、様々な前処理の影響を確認する

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import (
    ExtraTreesRegressor,
    HistGradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    RobustScaler,
    StandardScaler,
)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

fig, axes = plt.subplots(nrows=8, ncols=5, figsize=(16, 30))
for i, (estimator_name, estimator) in enumerate(
    [
        ["PLS", PLSRegression],
        ["KN", KNeighborsRegressor],
        ["SVM", SVR],
        ["DT", DecisionTreeRegressor],
        ["RF", RandomForestRegressor],
        ["ET", ExtraTreesRegressor],
        ["GB", HistGradientBoostingRegressor],
        ["MLP", MLPRegressor],
    ]
):
    for j, (scaler_name, scaler) in enumerate(
        [
            ["None", None],
            ["MaxAbs", MaxAbsScaler],
            ["MinMax", MinMaxScaler],
            ["Robust", RobustScaler],
            ["Standard", StandardScaler],
        ]
    ):

        if scaler is None:
            model = estimator()
        else:
            model = Pipeline([(scaler_name, scaler()), (estimator_name, estimator())])
        model.fit(X_train, Y1_train)
        score = model.score(X_test, Y1_test)
        axes[i][j].set_title(
            "{0} {1} {2:.3f}".format(scaler_name, estimator_name, score)
        )
        axes[i][j].scatter(Y1_test, model.predict(X_test), c=Y1_test)
        axes[i][j].plot(Y1_test, Y1_test)

# PCAで主成分を抜き出して様々な回帰モデルへの影響を確認する

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    ExtraTreesRegressor,
    HistGradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    RobustScaler,
    StandardScaler,
)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

fig, axes = plt.subplots(nrows=8, ncols=5, figsize=(16, 30))
for i, (estimator_name, estimator) in enumerate(
    [
        ["PLS", PLSRegression],
        ["KN", KNeighborsRegressor],
        ["SVM", SVR],
        ["DT", DecisionTreeRegressor],
        ["RF", RandomForestRegressor],
        ["ET", ExtraTreesRegressor],
        ["GB", HistGradientBoostingRegressor],
        ["MLP", MLPRegressor],
    ]
):
    for j, n_components in enumerate([2, 50, 100, 200, 400]):

        model = Pipeline(
            [
                ("MinMax", MinMaxScaler()),
                ("PCA", PCA(n_components=n_components)),
                (estimator_name, estimator()),
            ]
        )
        model.fit(X_train, Y1_train)
        score = model.score(X_test, Y1_test)
        axes[i][j].set_title(
            "{0} {1} {2:.3f}".format(n_components, estimator_name, score)
        )
        axes[i][j].scatter(Y1_test, model.predict(X_test), c=Y1_test)
        axes[i][j].plot(Y1_test, Y1_test)

# Tree系回帰モデル・分類モデルで特徴量の重要度を確認する

In [ ]:
from sklearn.ensemble import (
    ExtraTreesClassifier,
    ExtraTreesRegressor,
    GradientBoostingClassifier,
    GradientBoostingRegressor,
    RandomForestClassifier,
    RandomForestRegressor,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(16, 16))
for i, (estimator_name, regressor, classifier) in enumerate(
    [
        ["DT", DecisionTreeRegressor, DecisionTreeClassifier],
        ["RF", RandomForestRegressor, RandomForestClassifier],
        ["ET", ExtraTreesRegressor, ExtraTreesClassifier],
        ["GB", GradientBoostingRegressor, GradientBoostingClassifier],
    ]
):
    for j, (estimator, task, X_tr, X_te, Y_tr, Y_te) in enumerate(
        [
            [regressor, "Regr", X_train, X_test, Y1_train, Y1_test.values.tolist()],
            [classifier, "Clf", X_train, X_test, Y2_train, Y2_test.values.tolist()],
            [classifier, "Clf", X_train, X_test, Y3_train, Y3_test.values.tolist()],
        ]
    ):
        model = Pipeline(
            [
                ("MinMax", MinMaxScaler()),
                (estimator_name, estimator()),
            ]
        )
        model.fit(X_tr, Y_tr)
        score = model.score(X_te, Y_te)
        axes[i][j].set_title("{0} {1} {2:.3f}".format(estimator_name, task, score))
        topnum = 10
        importances = sorted(
            [(fi, name) for fi, name in zip(model[1].feature_importances_, X.columns)]
        )[::-1]
        axes[i][j].barh(
            [str(x[1]) for x in importances[:topnum][::-1]],
            [x[0] for x in importances[:topnum][::-1]],
        )

## 回帰モデルの重要度の算出

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train, Y1_train)
importances = sorted(
    [(fi, name) for fi, name in zip(model.feature_importances_, X_train.columns)]
)[::-1]

# 重要な特徴量を抜き出して様々な回帰モデルへの影響を確認する（特徴選択）

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    ExtraTreesRegressor,
    HistGradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    RobustScaler,
    StandardScaler,
)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

fig, axes = plt.subplots(nrows=8, ncols=5, figsize=(16, 30))
for i, (estimator_name, estimator) in enumerate(
    [
        ["PLS", PLSRegression],
        ["KN", KNeighborsRegressor],
        ["SVM", SVR],
        ["DT", DecisionTreeRegressor],
        ["RF", RandomForestRegressor],
        ["ET", ExtraTreesRegressor],
        ["GB", HistGradientBoostingRegressor],
        ["MLP", MLPRegressor],
    ]
):
    for j, n_components in enumerate([2, 50, 100, 200, 400]):
        selected_columns = [name for fi, name in importances[:n_components]]
        model = Pipeline(
            [
                ("MinMax", MinMaxScaler()),
                (estimator_name, estimator()),
            ]
        )
        model.fit(X_train[selected_columns], Y1_train)
        score = model.score(X_test[selected_columns], Y1_test)
        axes[i][j].set_title(
            "{0} {1} {2:.3f}".format(n_components, estimator_name, score)
        )
        axes[i][j].scatter(Y1_test, model.predict(X_test[selected_columns]), c=Y1_test)
        axes[i][j].plot(Y1_test, Y1_test)

# 分類モデルの重要度の算出

In [ ]:
model = GradientBoostingClassifier()
model.fit(X_train, Y3_train)
importances = sorted(
    [(fi, name) for fi, name in zip(model.feature_importances_, X_train.columns)]
)[::-1]

# 重要な特徴量を抜き出して様々な分類モデルへの影響を確認する（特徴選択）

In [ ]:
from sklearn import metrics
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    ExtraTreesClassifier,
    HistGradientBoostingClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    RobustScaler,
    StandardScaler,
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor

fig, axes = plt.subplots(nrows=8, ncols=5, figsize=(16, 30))
for i, (estimator_name, estimator) in enumerate(
    [
        ["LR", LogisticRegression],
        ["KN", KNeighborsClassifier],
        ["SVM", SVC],
        ["DT", DecisionTreeClassifier],
        ["RF", RandomForestClassifier],
        ["ET", ExtraTreesClassifier],
        ["GB", HistGradientBoostingClassifier],
        ["MLP", MLPClassifier],
    ]
):
    for j, n_components in enumerate([2, 50, 100, 200, 400]):
        selected_columns = [name for fi, name in importances[:n_components]]
        model = Pipeline(
            [
                ("MinMax", MinMaxScaler()),
                (estimator_name, estimator()),
            ]
        )
        model.fit(X_train[selected_columns], Y3_train)
        score = model.score(X_test[selected_columns], Y3_test)
        axes[i][j].set_title(
            "{0} {1} {2:.3f}".format(n_components, estimator_name, score)
        )
        tn, fp, fn, tp = metrics.confusion_matrix(
            Y3_test, model.predict(X_test[selected_columns])
        ).ravel()
        # axes[i][j].set_title("tn, fp, fn, tp = {} {} {} {}".format(tn, fp, fn, tp))
        axes[i][j].bar(["Positive", "Negative"], [tp, tn])
        axes[i][j].bar(["Positive", "Negative"], [-fn, -fp])
        axes[i][j].grid()

# Ensemble （アンサンブル）

- voting (複数の学習器を並列に用いて、平等に取り扱う)
- stacking (複数の学習器を並列に用いて、その結果をさらに学習する)
- bagging (同タイプの学習器を並列に用いて、異なるサンプリングデータで複数回学習する)
- boosting (同タイプの学習器を直列に用いて、前の学習器が間違えた部分を後の学習器が再学習する)

## Voting

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

model = VotingRegressor(
    [("svm", SVR()), ("kn", KNeighborsRegressor()), ("mlp", MLPRegressor())]
)
model.fit(X_train, Y1_train)
depict.regression_metrics(model, X_test, Y1_test)

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

model = VotingClassifier(
    [("svm", SVC()), ("kn", KNeighborsClassifier()), ("mlp", MLPClassifier())]
)
model.fit(X_train, Y3_train)
depict.classification_metrics(model, X_test, Y3_test)

## Stacking

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

model = StackingRegressor(
    [("svm", SVR()), ("kn", KNeighborsRegressor()), ("mlp", MLPRegressor())],
    final_estimator=RidgeCV(),
)
model.fit(X_train, Y1_train)
depict.regression_metrics(model, X_test, Y1_test)

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

model = StackingClassifier(
    [("svm", SVC()), ("kn", KNeighborsClassifier()), ("mlp", MLPClassifier())],
    final_estimator=LogisticRegression(),
)
model.fit(X_train, Y3_train)
depict.classification_metrics(model, X_test, Y3_test)

## Bagging

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor

model = BaggingRegressor(base_estimator=KNeighborsRegressor())
model.fit(X_train, Y1_train)
depict.regression_metrics(model, X_test, Y1_test)

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

model = BaggingClassifier(base_estimator=KNeighborsClassifier())
model.fit(X_train, Y3_train)
depict.classification_metrics(model, X_test, Y3_test)

## Boosting

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPClassifier

model = AdaBoostRegressor(base_estimator=KNeighborsRegressor())
model.fit(X_train, Y1_train)
depict.regression_metrics(model, X_test, Y1_test)

In [ ]:
from rdkit_supporter import depict
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

model = AdaBoostClassifier(base_estimator=RandomForestClassifier())
model.fit(X_train, Y3_train)
depict.classification_metrics(model, X_test, Y3_test)

# Optuna による多目的最適化ハイパーパラメーターチューニング

In [ ]:
dateflag = "0307c"
learning_time_limit = 600
timeout_optuna = 60000
n_trials_optuna = 1000
MODEL_PATH = "."

In [ ]:
from functools import wraps


# 学習に時間がかかりすぎる場合に強制終了するための方法
def on_timeout(limit, handler, hint=None):
    def notify_handler(signum, frame):
        handler(
            "'%s' terminated since it did not finish in %d seconds." % (hint, limit)
        )

    def __decorator(function):
        def __wrapper(*args, **kwargs):
            import signal

            signal.signal(signal.SIGALRM, notify_handler)
            signal.alarm(limit)
            result = function(*args, **kwargs)
            signal.alarm(0)
            return result

        return wraps(function)(__wrapper)

    return __decorator


def handler_func(msg):
    print(msg)

In [ ]:
import copy
import time

from sklearn.metrics import matthews_corrcoef, precision_score, r2_score, recall_score


# Optunaでチューニングするための基本クラス
class BestTune:
    def __init__(self, x_train, x_valid, t_train, t_valid, task="regressor"):
        # 訓練データを格納
        self.x_train = x_train
        self.t_train = t_train

        # 検証データを格納
        self.x_valid = x_valid
        self.t_valid = t_valid

        # regressor か classifier か
        self.task = task
        if self.task[0] == "r" or self.task[0] == "R":
            self.measure = r2_score
        else:
            self.measure = matthews_corrcoef

        # ベストモデルとスコアを格納
        self.best_score = None
        self.best_estimator_ = None

    def get_params(self, trial):
        raise NotImplementedError()

    def get_base_model(self):
        raise NotImplementedError()

    @on_timeout(limit=learning_time_limit, handler=handler_func, hint=u"BestTune")
    def fit(self, trial):
        model = self.get_base_model()(**self.get_params(trial))
        model.fit(self.x_train, self.t_train)
        return model

    def __call__(self, trial):
        if self.task == "precision_recall":
            start_time = time.perf_counter()
            # 教師データで学習
            model = self.fit(trial)

            # 検証データの予測性能を評価
            precision = precision_score(model.predict(self.x_valid), self.t_valid)
            recall = recall_score(model.predict(self.x_valid), self.t_valid)
            end_time = time.perf_counter()

            # 多目的最適化
            return precision, recall, end_time - start_time

        else:
            start_time = time.perf_counter()
            # 教師データで学習
            model = self.fit(trial)

            # 検証データの予測性能を評価
            score = self.measure(model.predict(self.x_valid), self.t_valid)
            end_time = time.perf_counter()

            # ベストスコアが出れば、そのベストモデルを記録
            if self.best_estimator_ is None or self.best_score < score:
                self.best_score = score
                self.best_estimator_ = copy.deepcopy(model)

            # 多目的最適化
            return max(-1, score), end_time - start_time

In [ ]:
# GradientBoosting

from sklearn.ensemble import (
    HistGradientBoostingClassifier,
    HistGradientBoostingRegressor,
)


class tune_GB(BestTune):
    def get_base_model(self):
        if self.task[0] == "r" or self.task[0] == "R":
            return HistGradientBoostingRegressor
        else:
            return HistGradientBoostingClassifier

    def default_params(self):
        params = {
            "max_depth": 100,
            "min_samples_leaf": 10,
            "learning_rate": 0.1,
            "max_iter": 100,
            "max_leaf_nodes": 31,
            "l2_regularization": 0.0,
            "max_bins": 255,
            "early_stopping": "auto",
            "scoring": "loss",
        }
        if self.task[0] == "r" or self.task[0] == "R":
            params["loss"] = "squared_error"
        else:
            params["loss"] = "log_loss"
        return params

    def get_params(self, trial):
        params = {}
        params["learning_rate"] = trial.suggest_float("learning_rate", 0.001, 0.1)
        params["max_iter"] = trial.suggest_int("max_iter", 10, 1000)
        params["max_leaf_nodes"] = trial.suggest_int("max_leaf_nodes", 10, 100)
        params["max_depth"] = trial.suggest_int("max_depth", 1, 200)
        params["min_samples_leaf"] = trial.suggest_int("min_samples_leaf", 1, 100)
        params["l2_regularization"] = trial.suggest_float("l2_regularization", 0, 10)
        params["max_bins"] = trial.suggest_int("max_bins", 2, 255)
        params["interaction_cst"] = trial.suggest_categorical(
            "interaction_cst", ["pairwise", "no_interactions"]
        )
        if self.task[0] == "r" or self.task[0] == "R":
            params["loss"] = trial.suggest_categorical(
                "loss", ["squared_error", "absolute_error"]
            )
        else:
            params["loss"] = trial.suggest_categorical(
                "loss", ["auto", "binary_crossentropy", "log_loss"]
            )
        return params

In [ ]:
import optuna


# Optuna で学習を繰り返し、学習履歴を保存する
def train(
    study_name,
    tune_model,
    timeout=timeout_optuna,
    n_trials=n_trials_optuna,
    show_progress_bar=True,
):
    import warnings

    warnings.simplefilter("ignore")
    optuna.logging.set_verbosity(optuna.logging.WARN)

    # 学習環境を立ち上げる
    study = optuna.create_study(
        study_name=study_name,
        storage="sqlite:///" + study_name + ".sql",
        load_if_exists=True,
        directions=["maximize", "maximize", "minimize"],
        sampler=optuna.samplers.NSGAIISampler(),
    )

    try:
        study.enqueue_trial(study.best_trial.params)
    except:
        try:
            study.enqueue_trial(tune_model.default_params())
        except:
            pass

    # 学習する
    study.optimize(
        tune_model,
        timeout=timeout,
        n_trials=n_trials,
        show_progress_bar=show_progress_bar,
    )
    return study

In [ ]:
strage_name = "GBR_{}".format(dateflag)
study = train(
    "{}{}_bestfit".format(MODEL_PATH, strage_name),
    tune_GB(X_train, X_test, Y3_train, Y3_test, task="precision_recall"),
)

## 学習履歴の表示（すべてのトライアル）

In [ ]:
results = []
for trial in study.trials:
    if trial.values is not None:
        params = trial.params
        params["trial_number"] = trial.number
        params["precision"] = trial.values[0]
        params["recall"] = trial.values[1]
        params["time"] = trial.values[2]
        results.append(params)

In [ ]:
for xlabel in sorted(params.keys()):
    if xlabel == "trial_number":
        continue
    for ylabel in ["precision", "recall", "time"]:
        if ylabel == "trial_number":
            continue
        if xlabel == ylabel:
            continue
        plt.scatter(
            [record[xlabel] for record in results],
            [record[ylabel] for record in results],
            c=[record["trial_number"] for record in results],
            cmap="Blues",
        )
        plt.grid()
        plt.colorbar()
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.show()

## 学習履歴の表示（ベストトライアル）

In [ ]:
results = []
for trial in study.best_trials:
    if trial.values is not None:
        params = trial.params
        params["trial_number"] = trial.number
        params["precision"] = trial.values[0]
        params["recall"] = trial.values[1]
        params["time"] = trial.values[2]
        results.append(params)

In [ ]:
for xlabel in sorted(params.keys()):
    if xlabel == "trial_number":
        continue
    for ylabel in ["precision", "recall", "time"]:
        if ylabel == "trial_number":
            continue
        if xlabel == ylabel:
            continue
        plt.scatter(
            [record[xlabel] for record in results],
            [record[ylabel] for record in results],
            c=[record["trial_number"] for record in results],
            cmap="Blues",
        )
        plt.grid()
        plt.colorbar()
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.show()